In [1]:
import pandas as pd
from collections import defaultdict
from difflib import SequenceMatcher
import difflib
from unidecode import  unidecode
from urllib.parse import unquote
import pickle

"""
input: year
Read the two pkl files
data: Contains the lineups from the match day
fifa: Contains the GES values of the players of the teams on matchday
ouput: data, fifa
"""
def einlesen(jahr):
    daten=pd.read_pickle("AufstellungenNeu"+str(jahr)+".pkl")
    fifa=pd.read_pickle("Fifa"+str(jahr)+".pkl")
    fifa=fifa.iloc[::-1]
    return daten,fifa

"""
input: data
create dict which creates per lineup on each matchday 
creates the lineup+team name
output: matchday data
"""
def spieltag_daten_erstellen(daten):
    #Nummer 33= 1.Spieltag,..., Nummer 0= 34.Spieltag
    #print(daten)
    spieltag_daten={}
    for spieltagsnummer, mannschaftsdaten in daten.items():
        
        #print(mannschaftsdaten)
        spielerliste=[]
        count=1
        for mannschaft, spieler in mannschaftsdaten.items():
            #print(mannschaft)
            spieler.append(mannschaft)
            spielerliste.append(spieler)
        spieltag_daten.update({spieltagsnummer:spielerliste})
    return spieltag_daten
"""
input: fifa
get GES values+ player name, team name
output: dic
"""
def spieltag_fifa_erstellen(fifa):
    liga=[]
    teamnames_fifa=[]
    for i in fifa.to_dict().items():
        team=i[0]
        liste= i[1]
        for j in range(33,-1,-1):
            listehelp= liste[j]
            listehelp.append(team)
            liga.append({j:listehelp})


    spieltag_fifa={}

    for i in range(33,-1,-1):
        zahl=i%33
        mannschaften=[]
        for mannschaft in liga:
            for spieltage,spieler in mannschaft.items():
                if spieltage==i:
                    #print(spieler)
                    mannschaften.append(spieler)

        spieltag_fifa.update({i:mannschaften})
    return spieltag_fifa

"""
input: team, list of teams
Similarity search, which team will be considered on match day
output: team 
"""  
def similar(mannschaft, liste): 
    wert=0
    for team in liste:
        similarity= SequenceMatcher(None, mannschaft, team).ratio()
        #print(team, similarity)
        if wert < similarity:
            wert= similarity
            currentteam= team
            
    return currentteam

"""
input: matchday data, fifa data for match day
Find team which is considered
save player values and team name
for each game per matchday and for each matchday per season
output: dict
"""
def result_erstellen(spieltag_daten,spieltag_fifa):
    mannschaften=[] 
    for mannschaft in spieltag_fifa[1]: 
        team= mannschaft[len(mannschaft)-1] 
        team= unidecode(unquote(team)) 
        mannschaften.append(team)

    result={} 
    
    count=1 
    for key,value in spieltag_daten.items():
        mannschaften_daten=[]
        aufstellungen_daten=[]
        spieltag={}
        team_ges=[] 
        #Mannschaften aus daten sammeln
        for spieler in value:
            aufstellung= spieler
            aufstellungen_daten.append(aufstellung)
            mannschaft= spieler[len(spieler)-1]
            if str(mannschaft) == "hsv":
                mannschaft= "Hamburg"
            mannschaften_daten.append(mannschaft)
        
        #print(len(mannschaften_daten))
        for liste in spieltag_fifa[key-1]:
            mannschaften_ges=[]
            aufstellung_fifa= liste[:len(liste)-1]
            mannschaft2= liste[len(liste)-1]
            mannschaft2=unidecode(unquote(mannschaft2))
            
            current_mannschaft= similar(mannschaft2,mannschaften_daten)
            #print(mannschaft2," ",current_mannschaft,mannschaften_daten,"\n")
            index= mannschaften_daten.index(current_mannschaft)
            current_aufstellung= aufstellungen_daten[index]
            current_aufstellung= current_aufstellung[:-1]
            
            for spielername in current_aufstellung:
                name= similar(spielername, aufstellung_fifa)
                ges= name.split(" ")[len(name.split(" "))-1]
                mannschaften_ges.append(ges)
            
            #if len(mannschaften_ges) <11:
                
                #print(current_mannschaft,mannschaften_ges)
                #print(aufstellungen_daten)
            spieltag.update({current_mannschaft:mannschaften_ges[:11]})
            
            team_ges.append(spieltag)
        
        
        result.update({count:team_ges[-1]})
        count +=1
        

    return result

#dict in pkl speichern
"""
input: dict, year
save data in pkl file
output: pkl file
"""
def pkl_erstellen(result,jahr):
    data= pd.DataFrame.from_dict(result)
    filename= "GES"+str(jahr)+".pkl"
    file = open(filename, 'wb')
    pickle.dump(data,file)
    file.close()

